# Worksheet 08

Name: Dia Villalpando

UID: U70784338

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [94]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.194777891070683, 7.5354145408385405, 4.987009332906668, 4.891772899692095, 4.4793401879878285, 5.992101389652031, 6.324682534656361, 5.229692186689191, 4.119133631542454, 5.311222608428785]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [95]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.960560235218748, 7.485851208907818, 6.409014436612152, 8.356055784094282, 9.308239952182236, 7.162901472853386, 8.264190259915113, 7.970600484661782, 8.81809715488089, 8.719595272449599]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [96]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.311222608428785, 4.119133631542454, 5.229692186689191, 8.719595272449599, 6.324682534656361, 5.992101389652031, 8.81809715488089, 4.4793401879878285, 7.970600484661782, 8.264190259915113]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Prior Probability = 50%
- Mean for S1 = 5
- Variance for S1 = 1
- Mean for S2 = 8
- Variance for S2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [97]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s2) + len(s1)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , 
       sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.311222608428785, 4.119133631542454, 5.229692186689191, 6.324682534656361, 5.992101389652031, 4.4793401879878285]
[8.719595272449599, 8.81809715488089, 7.970600484661782, 8.264190259915113]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.242695423159442,  mean_2 = 8.443120792976845
var_1 = 0.5970454643308775,  var_2 = 0.11808424680461041


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [98]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0]*prob_s[0])/prob_x )
    prob_s1_x.append( (pdf_i[1]*prob_s[1])/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.311222608428785
probability of observing that point if it came from cluster 0 =  0.6638072763609609
probability of observing that point if it came from cluster 1 =  5.986275969580693e-153
point =  4.119133631542454
probability of observing that point if it came from cluster 0 =  0.11373394487897692
probability of observing that point if it came from cluster 1 =  2.3105254232109635e-291
point =  5.229692186689191
probability of observing that point if it came from cluster 0 =  0.6680356834855529
probability of observing that point if it came from cluster 1 =  5.25654226279485e-161
point =  8.719595272449599
probability of observing that point if it came from cluster 0 =  2.8888936168712675e-08
probability of observing that point if it came from cluster 1 =  0.21794764350867452
point =  6.324682534656361
probability of observing that point if it came from cluster 0 =  0.12934453055461226
probability of observing that point if it came from cluster 1 =  4.373357856052525e-70
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [99]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), 
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

# var = [ ... , ... ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , 
       sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.254201686385922,  mean_2 = 8.446141825378579
var_1 = 0.5971778584243146,  var_2 = 0.11809337344138274


A very slight change, it feels like the means and variances all increased very slightly

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [86]:
k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0]*prob_s[0])/prob_x )
    prob_s1_x.append( (pdf_i[1]*prob_s[1])/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.791812535574289
probability of observing that point if it came from cluster 0 =  0.5373403258184521
probability of observing that point if it came from cluster 1 =  2.399469134790143e-07
point =  8.724954520072902
probability of observing that point if it came from cluster 0 =  2.867023494474118e-07
probability of observing that point if it came from cluster 1 =  0.5581570442553528
point =  3.866642330500845
probability of observing that point if it came from cluster 0 =  0.26752587913720716
probability of observing that point if it came from cluster 1 =  8.572828332593141e-11
point =  4.915253969867861
probability of observing that point if it came from cluster 0 =  0.5241392858804135
probability of observing that point if it came from cluster 1 =  6.079844462946969e-07
point =  7.589961458244568
probability of observing that point if it came from cluster 0 =  0.00033431763686533867
probability of observing that point if it came from cluster 1 =  0.19360645770287463
point =

it looks like the values are less certain

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [101]:
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    if(p[1]>p[2]):
        print("Comes from S_1")
    else:
        print("Comes from S_2")

5.311222608428785
Comes from S_1
4.119133631542454
Comes from S_1
5.229692186689191
Comes from S_1
8.719595272449599
Comes from S_2
6.324682534656361
Comes from S_1
5.992101389652031
Comes from S_1
8.81809715488089
Comes from S_2
4.4793401879878285
Comes from S_1
7.970600484661782
Comes from S_2
8.264190259915113
Comes from S_2
